In [36]:
r = requests.get('https://spotifycharts.com/regional/tw/weekly/2020-11-13--2020-11-20')
soup = BeautifulSoup(r.text, 'html.parser')
len(soup.find('div', {'class': 'chart-filters-list'}).find('div', {'data-type':'country'}).find('ul').findAll('li'))
countries = [(c['data-value'], c.text) for c in soup.find('div', {'class': 'chart-filters-list'}).find('div', {'data-type':'country'}).find('ul').findAll('li')]
countries

[('global', 'Global'),
 ('us', 'United States'),
 ('gb', 'United Kingdom'),
 ('ad', 'Andorra'),
 ('ar', 'Argentina'),
 ('at', 'Austria'),
 ('au', 'Australia'),
 ('be', 'Belgium'),
 ('bg', 'Bulgaria'),
 ('bo', 'Bolivia'),
 ('br', 'Brazil'),
 ('ca', 'Canada'),
 ('ch', 'Switzerland'),
 ('cl', 'Chile'),
 ('co', 'Colombia'),
 ('cr', 'Costa Rica'),
 ('cy', 'Cyprus'),
 ('cz', 'Czech Republic'),
 ('de', 'Germany'),
 ('dk', 'Denmark'),
 ('do', 'Dominican Republic'),
 ('ec', 'Ecuador'),
 ('ee', 'Estonia'),
 ('es', 'Spain'),
 ('fi', 'Finland'),
 ('fr', 'France'),
 ('gr', 'Greece'),
 ('gt', 'Guatemala'),
 ('hk', 'Hong Kong'),
 ('hn', 'Honduras'),
 ('hu', 'Hungary'),
 ('id', 'Indonesia'),
 ('ie', 'Ireland'),
 ('il', 'Israel'),
 ('in', 'India'),
 ('is', 'Iceland'),
 ('it', 'Italy'),
 ('jp', 'Japan'),
 ('lt', 'Lithuania'),
 ('lu', 'Luxembourg'),
 ('lv', 'Latvia'),
 ('mx', 'Mexico'),
 ('my', 'Malaysia'),
 ('ni', 'Nicaragua'),
 ('nl', 'Netherlands'),
 ('no', 'Norway'),
 ('nz', 'New Zealand'),
 ('pa', '

查看總共有幾週

In [2]:
import requests
from bs4 import BeautifulSoup
r = requests.get('https://spotifycharts.com/regional/tw/weekly/2020-11-13--2020-11-20')
soup = BeautifulSoup(r.text, 'html.parser')
len(soup.find('div', {'class': 'chart-filters-list'}).find('div', {'data-type':'date'}).find('ul').findAll('li'))

205

In [24]:
import requests
from bs4 import BeautifulSoup
import datetime as dt
import pandas as pd

# 要爬的網址、api
spotify = 'https://spotifycharts.com/'
track_api = 'https://api.spotify.com/v1/tracks'
feature_api = 'https://api.spotify.com/v1/audio-features'
artists_api = 'https://api.spotify.com/v1/artists'

# 設定要爬的國家
# regions = [('de', 'Germany'), ('it', 'Italy'), ('es', 'Spain'), ('nl', 'Netherlands'), ('fr', 'France'), ('fi', 'Finland')]
# regions = [ ('us', 'United States'), ('gb', 'United Kingdom'), ('at', 'Austria'), ('au', 'Australia'), ('br', 'Brazil')]
regions = [('au', 'Australia'), ('br', 'Brazil')]
for region, country in regions:
    
    time_start = dt.datetime.now()
    print('===================================================')
    print(f'{country} start!!')
    
    '''
    爬每週 top200 的名單
    '''
    fri = dt.date(2020, 11, 27)
    track_ids, w = [], 0
    while w < 205:  # 總共有205週可以爬
        end, start = str(fri - dt.timedelta(days = 7*w)), str(fri - dt.timedelta(days = 7*(w+1)))
        week = f'{start}--{end}'
        w += 1  

        # 爬當週的top200名單
        url = f'{spotify}regional/{region}/weekly/{week}/download' 
        r = requests.get(url)
        table = r.text.split('\n')[2:-1]
        
        # 整理 top200 track_ids
        _ = [track.split(',')[-1].split('/')[-1] for track in table]
        track_ids += _
        print(f'取得week{str(w).zfill(3)} id')
    
    track_ids = list(set(track_ids)) # 只取不重複的歌
    print(f'{country}共有{len(track_ids)}首歌')
    
    '''
    爬 track_api 和 feature_api
    '''
    
    CLIENT_ID = 'ad0d8699855a4e72b183657dc0f7d55a'
    CLIENT_SECRET = '3e66a60843e34360add3d4246c34fc52'
    AUTH_URL = 'https://accounts.spotify.com/api/token'
    auth_response = requests.post(AUTH_URL, {'grant_type': 'client_credentials', 'client_id': CLIENT_ID, 'client_secret': CLIENT_SECRET,})
    # 獲得 access_token (時間到會失效，每爬一個國家就重取一次token)
    access_token = auth_response.json()['access_token']
    headers = {'Authorization': f'Bearer {access_token}'}
    
    data = pd.DataFrame(columns = ['track', 'feature'])
    track, feature = [], []
    i = 0 
    while i < len(track_ids):
        ids_t = ','.join(track_ids[i : i+50])  # tracks 一次最多50筆
        track += requests.get(track_api, params = {'ids':ids_t}, headers = headers).json()['tracks']
        
        if i%100 == 0:
            ids_f = ','.join(track_ids[i:i+100])  # audio-features 一次最多100筆
            feature += requests.get(feature_api, params = {'ids':ids_f}, headers = headers).json()['audio_features']
        
        i += 50
        
    data = pd.concat([data, pd.DataFrame(zip(track, feature), columns = ['track', 'feature'])], axis = 0)

    '''
    把爬蟲爬到的東西分到各個column裡
    '''
    # track_api 爬到的東西
    _ = [('track_name', 'name'), ('track_id', 'id'), ('popularity', 'popularity')]
    for col, fea in _:
        data[col] = data['track'].apply(lambda x: x[fea])
    _ = [('album_name', 'name'), ('album_id', 'id'), ('release_date', 'release_date')] 
    for col, fea in _:
        data[col] = data['track'].apply(lambda x: x['album'][fea])
    _ = [('artist_name', 'name'), ('artist_id', 'id')]    
    for col, fea in _:
        data[col] = data['track'].apply(lambda x: x['artists'][0][fea])

    # feature_api 爬到的東西    
    features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature']
    for key in features:
        data[key] = data['feature'].apply(lambda x:x[key])
        
    print('取得track_api、feature_api資料')    

    '''
    從 artists_api 取得 genres 資訊
    '''    
    # 建立 artists_id、genres 對照表
    artist_genres = dict()
    artists_ids = list(data['artist_id'].unique())

    i = 0
    while i < len(artists_ids):
        ids = ','.join(artists_ids[i:i+50]) # artists 一次最多50筆

        r = requests.get(artists_api, params = {'ids':ids}, headers = headers)
        artist_genres.update({artist['id']:artist['genres'] for artist in r.json()['artists']})

        i += 50 

    # 取得 genres
    data['genres'] = data['artist_id'].map(artist_genres)
    print('取得genre資料')
    
    # 加上 country 資訊
    data['country'] = country

    print(dt.datetime.now() - time_start)

    data[['track', 'feature']].to_csv(f'/Users/shao/learning_python/ccClub advance/mid_project/track_feature_data/{country}.csv', index=False)
    data = data.drop(['track', 'feature'], axis = 1)
    data.to_csv(f'/Users/shao/learning_python/ccClub advance/mid_project/data/data_{country}.csv', index = False)


Australia start!!
取得week001 id
取得week002 id
取得week003 id
取得week004 id
取得week005 id
取得week006 id
取得week007 id
取得week008 id
取得week009 id
取得week010 id
取得week011 id
取得week012 id
取得week013 id
取得week014 id
取得week015 id
取得week016 id
取得week017 id
取得week018 id
取得week019 id
取得week020 id
取得week021 id
取得week022 id
取得week023 id
取得week024 id
取得week025 id
取得week026 id
取得week027 id
取得week028 id
取得week029 id
取得week030 id
取得week031 id
取得week032 id
取得week033 id
取得week034 id
取得week035 id
取得week036 id
取得week037 id
取得week038 id
取得week039 id
取得week040 id
取得week041 id
取得week042 id
取得week043 id
取得week044 id
取得week045 id
取得week046 id
取得week047 id
取得week048 id
取得week049 id
取得week050 id
取得week051 id
取得week052 id
取得week053 id
取得week054 id
取得week055 id
取得week056 id
取得week057 id
取得week058 id
取得week059 id
取得week060 id
取得week061 id
取得week062 id
取得week063 id
取得week064 id
取得week065 id
取得week066 id
取得week067 id
取得week068 id
取得week069 id
取得week070 id
取得week071 id
取得week072 id
取得week073 id
取得week074 id
取得week075 id
取得week0

In [20]:
# requests.get(track_api, params = {'ids':ids_t}, headers = headers).json()['tracks']
data[['track', 'feature']]

,track,feature
0,"{'album': {'album_type': 'album', 'artists': [...","{'danceability': 0.358, 'energy': 0.557, 'key'..."
1,"{'album': {'album_type': 'album', 'artists': [...","{'danceability': 0.762, 'energy': 0.426, 'key'..."
2,"{'album': {'album_type': 'album', 'artists': [...","{'danceability': 0.448, 'energy': 0.824, 'key'..."
3,"{'album': {'album_type': 'single', 'artists': ...","{'danceability': 0.822, 'energy': 0.743, 'key'..."
4,"{'album': {'album_type': 'album', 'artists': [...","{'danceability': 0.631, 'energy': 0.75, 'key':..."
...,...,...
435,"{'album': {'album_type': 'album', 'artists': [...","{'danceability': 0.688, 'energy': 0.647, 'key'..."
436,"{'album': {'album_type': 'album', 'artists': [...","{'danceability': 0.719, 'energy': 0.598, 'key'..."
437,"{'album': {'album_type': 'album', 'artists': [...","{'danceability': 0.963, 'energy': 0.449, 'key'..."
438,"{'album': {'album_type': 'album', 'artists': [...","{'danceability': 0.582, 'energy': 0.693, 'key'..."


In [9]:
pd.read_csv('/Users/shao/learning_python/ccClub advance/mid_project/data/data_Taiwan.csv')

,track,feature,track_name,track_id,popularity,album_name,album_id,release_date,artist_name,artist_id,...,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genres,country
0,"{'album': {'album_type': 'single', 'artists': ...","{'danceability': 0.425, 'energy': 0.201, 'key'...",刻在我心底的名字 (Your Name Engraved Herein) - 電影<刻在你心...,3OC84eKMxRJ4x0Hcwl9i4i,73,刻在我心底的名字 (Your Name Engraved Herein),5RG4bEVKGMdLaEIv1dofR2,2020-08-25,Crowd Lu,2JBUyLiFvpFPWdZGqIGYLD,...,136.328,audio_features,3OC84eKMxRJ4x0Hcwl9i4i,spotify:track:3OC84eKMxRJ4x0Hcwl9i4i,https://api.spotify.com/v1/tracks/3OC84eKMxRJ4...,https://api.spotify.com/v1/audio-analysis/3OC8...,320166,4,"['chinese indie', 'mandopop', 'taiwan indie', ...",Taiwan
1,"{'album': {'album_type': 'album', 'artists': [...","{'danceability': 0.398, 'energy': 0.557, 'key'...",我會一直想你,7wBMyVtVAUhuY02Tp2VU8n,32,你的名字,1Q22Xu4jKjLppEXQx7pWMt,2017-02-24,Jocie Guo,6hkP84fpANIA89Nv3T9JSm,...,171.963,audio_features,7wBMyVtVAUhuY02Tp2VU8n,spotify:track:7wBMyVtVAUhuY02Tp2VU8n,https://api.spotify.com/v1/tracks/7wBMyVtVAUhu...,https://api.spotify.com/v1/audio-analysis/7wBM...,302493,4,"['mandopop', 'singaporean mandopop']",Taiwan
2,"{'album': {'album_type': 'single', 'artists': ...","{'danceability': 0.558, 'energy': 0.58, 'key':...",Hat Trick,7rkrQeaED5Vwf0MllJS1jv,32,Hat Trick,0ZTjp0pn9RJ6JTnApPXncW,2019-01-24,Lexie Liu,6fs2or0cKLEM2xohWq8SoX,...,177.047,audio_features,7rkrQeaED5Vwf0MllJS1jv,spotify:track:7rkrQeaED5Vwf0MllJS1jv,https://api.spotify.com/v1/tracks/7rkrQeaED5Vw...,https://api.spotify.com/v1/audio-analysis/7rkr...,169109,3,['chinese electropop'],Taiwan
3,"{'album': {'album_type': 'album', 'artists': [...","{'danceability': 0.558, 'energy': 0.766, 'key'...",Sober,3qk4wRMDlaeagZGjSYbsOV,61,The Beautiful & Damned,1VAc77UvK5wj8ZSWCo3V2b,2017-12-15,G-Eazy,02kJSzxNuaWGqwubyUba0Z,...,96.281,audio_features,3qk4wRMDlaeagZGjSYbsOV,spotify:track:3qk4wRMDlaeagZGjSYbsOV,https://api.spotify.com/v1/tracks/3qk4wRMDlaea...,https://api.spotify.com/v1/audio-analysis/3qk4...,203960,4,"['hip hop', 'indie pop rap', 'oakland hip hop'...",Taiwan
4,"{'album': {'album_type': 'single', 'artists': ...","{'danceability': 0.621, 'energy': 0.333, 'key'...",祝你幸福,7iMCxA79b2oexaFVCj0Ml4,22,祝你幸福,4AHD6CIApXGHzYslqRkV4c,2017-10-10,Ronghao Li,0rTP0x4vRFSDbhtqcCqc8K,...,124.022,audio_features,7iMCxA79b2oexaFVCj0Ml4,spotify:track:7iMCxA79b2oexaFVCj0Ml4,https://api.spotify.com/v1/tracks/7iMCxA79b2oe...,https://api.spotify.com/v1/audio-analysis/7iMC...,277900,4,"['mainland chinese pop', 'mandopop']",Taiwan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4266,"{'album': {'album_type': 'album', 'artists': [...","{'danceability': 0.345, 'energy': 0.641, 'key'...",Dear Me,2izlCxatn0nR7SyRyrkUW1,52,Purpose - The 2nd Album Repackage,5IlYFprRPoMZ0qeNh6ghcs,2020-01-15,TAEYEON,3qNVuliS40BLgXGxhdBdqu,...,159.631,audio_features,2izlCxatn0nR7SyRyrkUW1,spotify:track:2izlCxatn0nR7SyRyrkUW1,https://api.spotify.com/v1/tracks/2izlCxatn0nR...,https://api.spotify.com/v1/audio-analysis/2izl...,222653,4,['k-pop'],Taiwan
4267,"{'album': {'album_type': 'album', 'artists': [...","{'danceability': 0.674, 'energy': 0.881, 'key'...",Don't Call Me Up,5WHTFyqSii0lmT9R21abT8,81,Ivy To Roses (Mixtape),0syM7OUAhV7S6XmOa4nLUZ,2019-01-18,Mabel,1MIVXf74SZHmTIp4V4paH4,...,98.994,audio_features,5WHTFyqSii0lmT9R21abT8,spotify:track:5WHTFyqSii0lmT9R21abT8,https://api.spotify.com/v1/tracks/5WHTFyqSii0l...,https://api.spotify.com/v1/audio-analysis/5WHT...,178480,4,"['dance pop', 'pop', 'pop dance', 'post-teen p...",Taiwan
4268,"{'album': {'album_type': 'single', 'artists': ...","{'danceability': 0.552, 'energy': 0.65, 'key':...",Sanctuary,3xaugmCyXrVkrDTXbFkMW3,73,Sanctuary,1cjgTku8zcYVHRi8QQH1XI,2019-06-14,Joji,3MZsBdqDrRTJihTHQrO6Dq,...,167.788,audio_features,3xaugmCyXrVkrDTXbFkMW3,spotify:track:3xaugmCyXrVkrDTXbFkMW3,https://api.spotify.com/v1/tracks/3xaugmCyXrVk...,https://api.spotify.com/v1/audio-analysis/3xau...,180

In [138]:
# url = 'https://api.spotify.com/v1/artists/16s0YTFcyjP4kgFwt7ktrY/albums'

# r = requests.get(url, headers = headers)
# album_ids = [album['id'] for album in r.json()['items']]

url = 'https://api.spotify.com/v1/artists'
ids = '16s0YTFcyjP4kgFwt7ktrY,2QcZxAgcs2I1q7CtCkl6MI'
r = requests.get(url, params = {'ids':ids}, headers = headers)

[(artist['name'], artist['genres']) for artist in r.json()['artists']]
#[(album['name'], album['genres']) for album in r.json()['albums']]

[('Mayday', ['c-pop', 'mandopop', 'taiwan pop']),
 ('Eason Chan', ['c-pop', 'cantopop', 'mandopop'])]

In [145]:
list(data['artist_id'].unique())

['2JBUyLiFvpFPWdZGqIGYLD',
 '3Nrfpe0tUJi4K4DXYWgMUX',
 '6f4srX54JFrLNK4aTJe2Sc',
 '0blbVefuxOGltDBa00dspv',
 '3TtgOeQcNkf9WVDA4xPBJM',
 '7n2wHs1TKAczGzO7Dd2rGr',
 '4TcOznbEZBqev21LzAH4KE',
 '6cMRDBCHMYjyCH2D0s6uzr',
 '66CXWjxzNUsdJxJ2JdwvnR',
 '5fEQLwq1BWWQNR8GzhOIvi',
 '6qqNVTkY8uBg9cP3Jd7DAH',
 '6cP65fCRmCJQejKYSnVb47',
 '3hhUgkTf3fFYGogFMbV5Wv',
 '7n2Ycct7Beij7Dj7meI4X0',
 '2qXGNIlmY3JrYkxOWyXZsd',
 '1uNFoZAHBGtllmzznpCI3s',
 '4MMQmzYiUiuD8VwjrJu3v6',
 '5YGY8feqx7naU7z4HrwZM6',
 '75CM5fojYdKYD0xYSFh22Z',
 '3dI4Io8XE33J2o04ZwjR0Y',
 '190bkHbFrRvEhcB7Zpuv3y',
 '1SZdTuxme3mOk0D9pUFrET',
 '41MozSoPIsD1dJM0CLPjZF',
 '4xlAHgZnzzPHtoKnev0gwJ',
 '3aIDSTKS9yH745GUQBxDcS',
 '2elBjNSdBE2Y3f0j1mjrql',
 '7aRC4L63dBn3CiLDuWaLSI',
 '6g641431O1Xkl7HAs2yFEg',
 '4kVy1sDTGmCWzxmzY6soNZ',
 '0XATRDCYuuGhk0oE7C0o5G',
 '6YVMFz59CuY7ngCxTxjpxE',
 '64KEffDW9EtZ1y2vBYgq8T',
 '4rdSHzO4enUlVxdQeHPGTp',
 '6taSlFSqCqAEUHOliFyHt7',
 '5SIsztzgW9ErZcWRnp2hdS',
 '6M2wZ9GZgrQXHCFfjv46we',
 '4GNC7GD6oZMSxPGyXy4MNB',
 

In [55]:
df = pd.read_csv('/Users/shao/learning_python/ccClub advance/mid_project/data/data_Taiwan.csv')
df['track'].loc[0]

"{'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2JBUyLiFvpFPWdZGqIGYLD'}, 'href': 'https://api.spotify.com/v1/artists/2JBUyLiFvpFPWdZGqIGYLD', 'id': '2JBUyLiFvpFPWdZGqIGYLD', 'name': 'Crowd Lu', 'type': 'artist', 'uri': 'spotify:artist:2JBUyLiFvpFPWdZGqIGYLD'}], 'available_markets': ['AD', 'AE', 'AL', 'AR', 'AT', 'AU', 'BA', 'BE', 'BG', 'BH', 'BO', 'BR', 'BY', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HR', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO', 'JP', 'KW', 'KZ', 'LB', 'LI', 'LT', 'LU', 'LV', 'MA', 'MC', 'MD', 'ME', 'MK', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'OM', 'PA', 'PE', 'PH', 'PL', 'PS', 'PT', 'PY', 'QA', 'RO', 'RS', 'RU', 'SA', 'SE', 'SG', 'SI', 'SK', 'SV', 'TH', 'TN', 'TR', 'TW', 'UA', 'US', 'UY', 'VN', 'XK', 'ZA'], 'external_urls': {'spotify': 'https://open.spotify.com/album/5RG4bEVKGMdLaEIv1dofR2'}, 'href': 'htt

In [48]:
data

,track,feature
0,"{'album': {'album_type': 'single', 'artists': ...","{'danceability': 0.7, 'energy': 0.722, 'key': ..."
1,"{'album': {'album_type': 'single', 'artists': ...","{'danceability': 0.798, 'energy': 0.62, 'key':..."
2,"{'album': {'album_type': 'single', 'artists': ...","{'danceability': 0.631, 'energy': 0.239, 'key'..."
3,"{'album': {'album_type': 'single', 'artists': ...","{'danceability': 0.734, 'energy': 0.874, 'key'..."
4,"{'album': {'album_type': 'album', 'artists': [...","{'danceability': 0.8, 'energy': 0.658, 'key': ..."
...,...,...
95,"{'album': {'album_type': 'single', 'artists': ...","{'danceability': 0.653, 'energy': 0.81, 'key':..."
96,"{'album': {'album_type': 'single', 'artists': ...","{'danceability': 0.728, 'energy': 0.426, 'key'..."
97,"{'album': {'album_type': 'single', 'artists': ...","{'danceability': 0.869, 'energy': 0.485, 'key'..."
98,"{'album': {'album_type': 'single', 'artists': ...","{'danceability': 0.929, 'energy': 0.723, 'key'..."


In [32]:
data.to_csv('/Users/shao/learning_python/ccClub advance/mid_project/data_tw.csv')

In [29]:
len(data.groupby('id'))

4271

In [317]:
headers = {'Authorization': f'Bearer {access_token}'}
r = requests.get('https://api.spotify.com/v1/tracks/77OJPbm053M9LA5Pqs2yOZ', headers=headers)

In [342]:
soup = BeautifulSoup(r.text, 'html.parser')
#soup.json()
list(r.json().keys())
print(r.json()['popularity'], r.json()['name'], r.json()['artists'][0]['name'])

57 諷刺的情書 Hebe Tien


In [236]:
soup = BeautifulSoup(r.text, 'html.parser')
soup

{
  "album" : {
    "album_type" : "album",
    "artists" : [ {
      "external_urls" : {
        "spotify" : "https://open.spotify.com/artist/14bJhryXGk6H6qlGzwj3W5"
      },
      "href" : "https://api.spotify.com/v1/artists/14bJhryXGk6H6qlGzwj3W5",
      "id" : "14bJhryXGk6H6qlGzwj3W5",
      "name" : "Hebe Tien",
      "type" : "artist",
      "uri" : "spotify:artist:14bJhryXGk6H6qlGzwj3W5"
    } ],
    "available_markets" : [ "AD", "AE", "AL", "AR", "AT", "AU", "BA", "BE", "BG", "BH", "BO", "BR", "BY", "CA", "CH", "CL", "CO", "CR", "CY", "CZ", "DE", "DK", "DO", "DZ", "EC", "EE", "EG", "ES", "FI", "FR", "GB", "GR", "GT", "HK", "HN", "HR", "HU", "ID", "IE", "IL", "IN", "IS", "IT", "JO", "JP", "KW", "KZ", "LB", "LI", "LT", "LU", "LV", "MA", "MC", "MD", "ME", "MK", "MT", "MX", "MY", "NI", "NL", "NO", "NZ", "OM", "PA", "PE", "PH", "PL", "PS", "PT", "PY", "QA", "RO", "RS", "RU", "SA", "SE", "SG", "SI", "SK", "SV", "TH", "TN", "TR", "TW", "UA", "US", "UY", "VN", "XK", "ZA" ],
    "extern

搜尋taiwan category

In [508]:
headers = {'Authorization': f'Bearer {access_token}'}
r = requests.get('https://api.spotify.com/v1/browse/categories?country=TW', headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')
soup


{
  "categories" : {
    "href" : "https://api.spotify.com/v1/browse/categories?country=TW&amp;offset=0&amp;limit=20",
    "items" : [ {
      "href" : "https://api.spotify.com/v1/browse/categories/toplists",
      "icons" : [ {
        "height" : 275,
        "url" : "https://t.scdn.co/media/derived/toplists_11160599e6a04ac5d6f2757f5511778f_0_0_275_275.jpg",
        "width" : 275
      } ],
      "id" : "toplists",
      "name" : "Top Lists"
    }, {
      "href" : "https://api.spotify.com/v1/browse/categories/at_home",
      "icons" : [ {
        "height" : null,
        "url" : "https://t.scdn.co/images/04da469dd7be4dab96659aa1fa9f0ac9.jpeg",
        "width" : null
      } ],
      "id" : "at_home",
      "name" : "At Home"
    }, {
      "href" : "https://api.spotify.com/v1/browse/categories/mandopop",
      "icons" : [ {
        "height" : null,
        "url" : "https://t.scdn.co/images/fd2cdde10ea04ccfb10c902274be8815.jpeg",
        "width" : null
      } ],
      "id" : "mandopo

In [102]:

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

# Track ID from the URI
artist_id = '7Dx7RhX0mFuXhCOUgB01uM'

# actual GET request with proper header
r = requests.get(BASE_URL + 'artists/' + artist_id + '/albums', headers=headers)


In [103]:
soup = BeautifulSoup(r.text, 'html.parser')
soup

{
  "href" : "https://api.spotify.com/v1/artists/7Dx7RhX0mFuXhCOUgB01uM/albums?offset=0&amp;limit=20&amp;include_groups=album,single,compilation,appears_on",
  "items" : [ {
    "album_group" : "album",
    "album_type" : "album",
    "artists" : [ {
      "external_urls" : {
        "spotify" : "https://open.spotify.com/artist/7Dx7RhX0mFuXhCOUgB01uM"
      },
      "href" : "https://api.spotify.com/v1/artists/7Dx7RhX0mFuXhCOUgB01uM",
      "id" : "7Dx7RhX0mFuXhCOUgB01uM",
      "name" : "JJ Lin",
      "type" : "artist",
      "uri" : "spotify:artist:7Dx7RhX0mFuXhCOUgB01uM"
    } ],
    "available_markets" : [ "AD", "AE", "AL", "AR", "AT", "AU", "BA", "BE", "BG", "BH", "BO", "BR", "BY", "CA", "CH", "CL", "CO", "CR", "CY", "CZ", "DE", "DK", "DO", "DZ", "EC", "EE", "EG", "ES", "FI", "FR", "GB", "GR", "GT", "HK", "HN", "HR", "HU", "ID", "IE", "IL", "IN", "IS", "IT", "JO", "JP", "KW", "KZ", "LB", "LI", "LT", "LU", "LV", "MA", "MC", "MD", "ME", "MK", "MT", "MX", "MY", "NI", "NL", "NO", "NZ

In [1]:
import pandas as pd

In [56]:
data = pd.read_csv('/Users/shao/learning_python/ccClub advance/mid_project/data.csv')

In [64]:
pd.set_option('display.max_rows', 100)
data.groupby('year').count()['id']
data

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.9950,['Carl Woitschach'],0.708,158648,0.1950,0,6KbQ3uYMLKb5jDxLF7wYDD,0.563000,10,0.1510,-12.428,1,Singende Bataillone 1. Teil,0,1928,0.0506,118.469,0.7790,1928
1,0.9940,"['Robert Schumann', 'Vladimir Horowitz']",0.379,282133,0.0135,0,6KuQTIu1KoTTkLXKrwlLPV,0.901000,8,0.0763,-28.454,1,"Fantasiestücke, Op. 111: Più tosto lento",0,1928,0.0462,83.972,0.0767,1928
2,0.6040,['Seweryn Goszczyński'],0.749,104300,0.2200,0,6L63VW0PibdM1HDSBoqnoM,0.000000,5,0.1190,-19.924,0,Chapter 1.18 - Zamek kaniowski,0,1928,0.9290,107.177,0.8800,1928
3,0.9950,['Francisco Canaro'],0.781,180760,0.1300,0,6M94FkXd15sOAOQYRnWPN8,0.887000,1,0.1110,-14.734,0,Bebamos Juntos - Instrumental (Remasterizado),0,1928-09-25,0.0926,108.003,0.7200,1928
4,0.9900,"['Frédéric Chopin', 'Vladimir Horowitz']",0.210,687733,0.2040,0,6N6tiFZ9vLTSOIxkj8qKrd,0.908000,11,0.0980,-16.829,1,"Polonaise-Fantaisie in A-Flat Major, Op. 61",1,1928,0.0424,62.149,0.0693,1928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169904,0.1730,"['DripReport', 'Tyga']",0.875,163800,0.4430,1,4KppkflX7I3vJQk7urOJaS,0.000032,1,0.0891,-7.461,1,Skechers (feat. Tyga) - Remix,75,2020-05-15,0.1430,100.012,0.3060,2020
169905,0.0167,"['Leon Bridges', 'Terrace Martin']",0.719,167468,0.3850,0,1ehhGlTvjtHo2e4xJFB0SZ,0.031300,8,0.1110,-10.907,1,Sweeter (feat. Terrace Martin),64,2020-06-08,0.0403,128.000,0.2700,2020
169906,0.5380,"['Kygo', 'Oh Wonder']",0.514,180700,0.5390,0,52eycxprLhK3lPcRLbQiVk,0.002330,7,0.1080,-9.332,1,How Would I Know,70,2020-05-29,0.1050,123.700,0.1530,2020
169907,0.0714,"['Cash Cash', 'Andy Grammer']",0.646,167308,0.7610,0,3wYOGJYD31sLRmBgCvWxa4,0.000000,1,0.2220,-2.557,1,I Found You,70,2020-02-28,0.0385,129.916,0.4720,2020


In [225]:
data.groupby('year')['id'].count()

year
1921     128
1922      72
1923     169
1924     237
1925     263
        ... 
2016    1969
2017    2000
2018    2000
2019    2000
2020    1756
Name: id, Length: 100, dtype: int64

In [56]:
data['artists']
#boo = data['artists'].apply(lambda x:x.replace('[', '').replace(']', '').replace("'", '').replace(' ', '').replace(',', '').replace('&', '').replace('.', '').replace('"', '').isalpha())
boo = data['artists'].apply(lambda x:'林' in x)
data[boo]

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
52457,0.756,"['Vivian Chow', '林隆璇']",0.530,247467,0.404,0,1EUTgeF7oI8XE6AQSAAlV8,0.000000,2,0.132,-10.615,1,流言,37,1988,0.0353,65.930,0.150,1988
75377,0.581,['優客李林'],0.467,255547,0.436,0,7jqrgcmsmzJCPglfLEb3zG,0.000000,4,0.163,-7.225,1,輸了你贏了世界又如何,48,1994,0.0267,136.008,0.166,1994
131967,0.536,"['Tsai Chin', '余天', 'Julie Sue', '潘越雲', 'Jenny...",0.511,320720,0.547,0,5bcbAT32LJS87hklgOxUqM,0.000001,11,0.584,-10.864,1,明天會更好,47,1985-10-25,0.0469,74.622,0.372,1985


In [3]:
data_by_artist = pd.read_csv('/Users/shao/learning_python/ccClub advance/mid_project/data_by_artist.csv')
data_by_artist

,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count
0,"""Cats"" 1981 Original London Cast",0.575083,0.442750,247260.000000,0.386336,0.022717,0.287708,-14.205417,0.180675,115.983500,0.334433,38.000000,5,1,12
1,"""Cats"" 1983 Broadway Cast",0.862538,0.441731,287280.000000,0.406808,0.081158,0.315215,-10.690000,0.176212,103.044154,0.268865,33.076923,5,1,26
2,"""Fiddler On The Roof” Motion Picture Chorus",0.856571,0.348286,328920.000000,0.286571,0.024593,0.325786,-15.230714,0.118514,77.375857,0.354857,34.285714,0,1,7
3,"""Fiddler On The Roof” Motion Picture Orchestra",0.884926,0.425074,262890.962963,0.245770,0.073587,0.275481,-15.639370,0.123200,88.667630,0.372030,34.444444,0,1,27
4,"""Joseph And The Amazing Technicolor Dreamcoat""...",0.605444,0.437333,232428.111111,0.429333,0.037534,0.216111,-11.447222,0.086000,120.329667,0.458667,42.555556,11,1,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27616,鳳飛飛,0.884000,0.358000,259387.000000,0.208000,0.000002,0.150000,-9.524000,0.033900,131.261000,0.278000,35.000000,10,1,2
27617,黃品源,0.541000,0.578000,293840.000000,0.334000,0.000006,0.067500,-11.974000,0.026700,135.934000,0.243000,47.000000,9,0,2
27618,黃國隆,0.785455,0.570818,174582.727273,0.148400,0.000083,0.142191,-21.610091,0.054355,119.586273,0.741273,20.000000,5,1,11
27619,黃蜀娟,0.925143,0.322262,212989.857143,0.141490,0.000189,0.196757,-19.256714,0.055571,111.685381,0.267033,23.000000,10,1,42


In [10]:
# 比data_by_artist多一欄genres
data_w_genres = pd.read_csv('/Users/shao/learning_python/ccClub advance/mid_project/data_w_genres.csv')
data_w_genres

,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count,genres
0,"""Cats"" 1981 Original London Cast",0.575083,0.442750,247260.000000,0.386336,0.022717,0.287708,-14.205417,0.180675,115.983500,0.334433,38.000000,5,1,12,['show tunes']
1,"""Cats"" 1983 Broadway Cast",0.862538,0.441731,287280.000000,0.406808,0.081158,0.315215,-10.690000,0.176212,103.044154,0.268865,33.076923,5,1,26,[]
2,"""Fiddler On The Roof” Motion Picture Chorus",0.856571,0.348286,328920.000000,0.286571,0.024593,0.325786,-15.230714,0.118514,77.375857,0.354857,34.285714,0,1,7,[]
3,"""Fiddler On The Roof” Motion Picture Orchestra",0.884926,0.425074,262890.962963,0.245770,0.073587,0.275481,-15.639370,0.123200,88.667630,0.372030,34.444444,0,1,27,[]
4,"""Joseph And The Amazing Technicolor Dreamcoat""...",0.605444,0.437333,232428.111111,0.429333,0.037534,0.216111,-11.447222,0.086000,120.329667,0.458667,42.555556,11,1,9,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27616,鳳飛飛,0.884000,0.358000,259387.000000,0.208000,0.000002,0.150000,-9.524000,0.033900,131.261000,0.278000,35.000000,10,1,2,"['c-pop', 'classic mandopop', 'vintage chinese..."
27617,黃品源,0.541000,0.578000,293840.000000,0.334000,0.000006,0.067500,-11.974000,0.026700,135.934000,0.243000,47.000000,9,0,2,"['c-pop', 'classic cantopop', 'classic mandopop']"
27618,黃國隆,0.785455,0.570818,174582.727273,0.148400,0.000083,0.142191,-21.610091,0.054355,119.586273,0.741273,20.000000,5,1,11,[]
27619,黃蜀娟,0.925143,0.322262,212989.857143,0.141490,0.000189,0.196757,-19.256714,0.055571,111.685381,0.267033,23.000000,10,1,42,[]


In [41]:
data_by_genres = pd.read_csv('/Users/shao/learning_python/ccClub advance/mid_project/data_by_genres.csv')
data_by_genres[data_by_genres['genres'] == 'pop']

,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode
1957,pop,0.263629,0.629311,219246.866256,0.618522,0.023895,0.171738,-6.866721,0.087476,119.870033,0.486683,59.691127,7,1


In [9]:
data_by_year = pd.read_csv('/Users/shao/learning_python/ccClub advance/mid_project/data_by_year.csv')
data_by_year

,year,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode
0,1921,0.895823,0.425661,229911.914062,0.236784,0.322330,0.215814,-17.095437,0.077258,100.397758,0.425495,0.351562,7,1
1,1922,0.939236,0.480000,167904.541667,0.237026,0.440470,0.238647,-19.179958,0.115419,101.376139,0.534056,0.138889,10,1
2,1923,0.976329,0.568462,178356.301775,0.246936,0.401932,0.236656,-14.373882,0.098619,112.456598,0.624788,5.727811,0,1
3,1924,0.935575,0.548654,188461.649789,0.347033,0.583955,0.237875,-14.202304,0.090210,120.653359,0.668574,0.603376,10,1
4,1925,0.965422,0.571890,184130.699620,0.264373,0.408893,0.243094,-14.516707,0.115457,115.671715,0.616430,2.707224,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2016,0.280290,0.599976,219400.763840,0.592877,0.074646,0.180198,-7.949913,0.107298,119.070344,0.430769,61.371254,0,1
96,2017,0.289916,0.612286,209343.613000,0.586739,0.098209,0.194218,-8.422697,0.111752,116.840278,0.414465,64.861500,1,1
97,2018,0.271941,0.664930,200919.119000,0.590591,0.035948,0.171781,-7.253666,0.128140,122.004325,0.447141,67.276000,1,1
98,2019,0.289298,0.644215,197733.133000,0.578796,0.076518,0.167161,-8.041738,0.124799,118.868163,0.465856,69.655500,1,1
